In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
import math
import random
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score,roc_auc_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
data1 = pd.read_csv("../../dataset/NASA/PC1.csv")
data1.head()

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,10OComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,91,9,3,2,318,2089.21,0.04,27.68,75.47,57833.24,...,80,44,11,31,29,66,192,126,17,true
1,109,21,5,18,381,2547.56,0.04,28.37,89.79,72282.68,...,97,41,12,24,28,75,229,152,38,true
2,505,106,41,82,2339,20696.93,0.01,75.93,272.58,1571506.88,...,457,71,48,49,64,397,1397,942,178,true
3,107,25,7,14,619,4282.78,0.02,52.91,80.95,226588.75,...,103,32,4,39,35,86,359,260,40,true
4,74,11,1,8,294,1917.93,0.03,28.77,66.66,55178.46,...,60,71,14,49,29,63,169,125,21,true


In [3]:
data1.describe()

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,t,lOCode,10OComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount
count,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1.107000e+03,...,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000
mean,23.416441,5.518519,2.769648,3.325203,117.603433,700.973297,0.127570,15.422005,32.961843,2.887495e+04,...,1604.163948,22.472448,4.701897,0.943993,6.755194,13.329720,20.928636,66.611563,50.991870,9.591689
std,35.303118,8.965191,5.572331,6.407142,197.453254,1510.617359,0.140379,16.341026,35.393870,1.707934e+05,...,9488.523309,33.593166,10.527052,3.348170,12.310284,8.173119,29.065483,111.769537,86.360681,16.551783
min,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
25%,7.000000,1.000000,1.000000,1.000000,25.000000,97.670000,0.045000,5.710000,14.090000,5.627650e+02,...,31.265000,7.000000,0.000000,0.000000,1.000000,8.000000,6.000000,15.000000,10.000000,1.000000
50%,13.000000,3.000000,1.000000,2.000000,58.000000,276.600000,0.080000,11.670000,23.580000,3.192000e+03,...,177.330000,13.000000,0.000000,0.000000,3.000000,12.000000,12.000000,33.000000,24.000000,5.000000
75%,26.000000,6.000000,3.000000,3.000000,126.500000,678.135000,0.160000,20.510000,41.150000,1.240090e+04,...,688.935000,24.000000,5.000000,1.000000,8.000000,17.000000,25.000000,72.000000,56.000000,11.000000
max,602.000000,136.000000,123.000000,123.000000,2785.000000,25942.690000,2.000000,270.660000,598.330000,4.279633e+06,...,237757.400000,600.000000,159.000000,48.000000,225.000000,99.000000,538.000000,1641.000000,1144.000000,236.000000


In [4]:
data1.values.shape

(1107, 22)

In [5]:
data2=data1.values
for i in range(1107):
    if data2[i][21]=="true ":
        data2[i][21]=1
    else:
        data2[i][21]=0
data2[1]

array([109, 21, 5, 18, 381, 2547.56, 0.04, 28.37, 89.79, 72282.68, 0.85,
       4015.7, 97, 41, 12, 24, 28, 75, 229, 152, 38, 1], dtype=object)

In [6]:
X=np.array(data2).astype(float)  

In [7]:
X[:,:21]

array([[  91.,    9.,    3., ...,  192.,  126.,   17.],
       [ 109.,   21.,    5., ...,  229.,  152.,   38.],
       [ 505.,  106.,   41., ..., 1397.,  942.,  178.],
       ...,
       [   5.,    3.,    3., ...,   11.,    6.,    5.],
       [  18.,    8.,    5., ...,   61.,   50.,   15.],
       [  26.,   18.,   13., ...,  119.,  109.,   35.]])

In [8]:
y=X[:,21]

In [9]:
X=X[:,0:21]

In [10]:
from imblearn.over_sampling import RandomOverSampler,SMOTE
#os =  RandomOverSampler()
X_res, y_res = SMOTE().fit_resample(X, y)
X_res.shape,y_res.shape

((2062, 21), (2062,))

In [11]:
X_res.shape
#X_res=X_res.reshape(898,21,1)
#X_res.shape

(2062, 21)

In [12]:
import numpy as np
from sklearn import preprocessing
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import datetime

def show_accuracy(predictLabel,Label):
    Label = np.ravel(Label).tolist()
    predictLabel = predictLabel.tolist()
    count = 0
    for i in range(len(Label)):
        if Label[i] == predictLabel[i]:
            count += 1
    return (round(count/len(Label),5))

class node_generator(object):
    def __init__(self, whiten = False):
        self.Wlist = []
        self.blist = []
        self.function_num = 0
        self.whiten = whiten

    def sigmoid(self, x):
        return 1.0/(1 + np.exp(-x))

    def relu(self, x):
        return np.maximum(x, 0)

    def tanh(self, x):
        return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

    def linear(self, x):
        return x

    def orth(self, W):
        """
        目前看来，这个函数应该配合下一个generator函数是生成权重的
        """
        for i in range(0, W.shape[1]):
            w = np.mat(W[:,i].copy()).T
            w_sum = 0
            for j in range(i):
                wj = np.mat(W[:,j].copy()).T
                w_sum += (w.T.dot(wj))[0,0]*wj
            w -= w_sum
            w = w/np.sqrt(w.T.dot(w))
            W[:,i] = np.ravel(w)

        return W

    def generator(self, shape, times):
        for i in range(times):
            random.seed(i)
            W = 2*np.random.random(size=shape)-1
            if self.whiten == True:
                W = self.orth(W)   # 只在增强层使用
            b = 2*np.random.random() -1
            yield (W, b)

    def generator_nodes(self, data, times, batchsize, function_num):
        # 按照bls的理论，mapping layer是输入乘以不同的权重加上不同的偏差之后得到的
        # 若干组，所以，权重是一个列表，每一个元素可作为权重与输入相乘
        self.Wlist = [elem[0] for elem in self.generator((data.shape[1], batchsize), times)]
        self.blist = [elem[1] for elem in self.generator((data.shape[1], batchsize), times)]

        self.function_num = {'linear':self.linear,
                        'sigmoid': self.sigmoid,
                        'tanh':self.tanh,
                        'relu':self.relu }[function_num]  # 激活函数供不同的层选择
        # 下面就是先得到一组mapping nodes，再不断叠加，得到len(Wlist)组mapping nodes
        nodes = self.function_num(data.dot(self.Wlist[0]) + self.blist[0])
        for i in range(1, len(self.Wlist)):
            nodes = np.column_stack((nodes, self.function_num(data.dot(self.Wlist[i])+self.blist[i])))
        return nodes

    def transform(self,testdata):
        testnodes = self.function_num(testdata.dot(self.Wlist[0])+self.blist[0])
        for i in range(1,len(self.Wlist)):
            testnodes = np.column_stack((testnodes, self.function_num(testdata.dot(self.Wlist[i])+self.blist[i])))
        return testnodes

class scaler:
    def __init__(self):
        self._mean = 0
        self._std = 0
    
    def fit_transform(self,traindata):
        self._mean = traindata.mean(axis = 0)
        self._std = traindata.std(axis = 0)
        return (traindata-self._mean)/(self._std+0.001)
    
    def transform(self,testdata):
        return (testdata-self._mean)/(self._std+0.001)

class broadnet:
    def __init__(self, 
                 maptimes = 10, 
                 enhencetimes = 10,
                 map_function = 'linear',
                 enhence_function = 'linear',
                 batchsize = 'auto', 
                 reg = 0.001):
        
        self._maptimes = maptimes
        self._enhencetimes = enhencetimes
        self._batchsize = batchsize
        self._reg = reg
        self._map_function = map_function
        self._enhence_function = enhence_function
        
        self.W = 0
        self.pesuedoinverse = 0
        self.normalscaler = scaler()
        self.onehotencoder = preprocessing.OneHotEncoder(sparse = False)
        self.mapping_generator = node_generator()
        self.enhence_generator = node_generator(whiten = True)

    def fit(self,data,label,weight):
        if self._batchsize == 'auto':
            self._batchsize = data.shape[1]
        data = self.normalscaler.fit_transform(data)
        label = self.onehotencoder.fit_transform(np.mat(label).T)
        
        mappingdata = self.mapping_generator.generator_nodes(data,self._maptimes,self._batchsize,self._map_function)
        enhencedata = self.enhence_generator.generator_nodes(mappingdata,self._enhencetimes,self._batchsize,self._enhence_function)
        
        #print('number of mapping nodes {0}, number of enhence nodes {1}'.format(mappingdata.shape[1],enhencedata.shape[1]))
        #print('mapping nodes maxvalue {0} minvalue {1} '.format(round(np.max(mappingdata),5),round(np.min(mappingdata),5)))
        #print('enhence nodes maxvalue {0} minvalue {1} '.format(round(np.max(enhencedata),5),round(np.min(enhencedata),5)))
        
        inputdata = np.column_stack((mappingdata,enhencedata))
        pesuedoinverse = self.pinv(inputdata,self._reg,weight)
        self.W =  pesuedoinverse.dot(label)
        
        #print('W:', self.W)
        #print('W:', self.W.shape)  
    
    #改写伪逆矩阵算法，将权重输入
    def pinv(self,A,reg,weight):
        return np.mat(reg*np.eye(A.shape[1])+A.T.dot(weight).dot(A)).I.dot(A.T).dot(weight)
    
    def decode(self,Y_onehot):
        Y = []
        for i in range(Y_onehot.shape[0]):
            lis = np.ravel(Y_onehot[i,:]).tolist()
            Y.append(lis.index(max(lis)))
        return np.array(Y)
    
    def accuracy(self,predictlabel,label):
        label = np.ravel(label).tolist()
        predictlabel = predictlabel.tolist()
        count = 0
        for i in range(len(label)):
            if label[i] == predictlabel[i]:
                count += 1
        return (round(count/len(label),5))
        
    def predict(self,testdata):
        testdata = self.normalscaler.transform(testdata)
        test_mappingdata = self.mapping_generator.transform(testdata)
        test_enhencedata = self.enhence_generator.transform(test_mappingdata)
        
        test_inputdata = np.column_stack((test_mappingdata,test_enhencedata)) 
        #print('*predictlabel shape:',self.decode(test_inputdata.dot(self.W)).shape)
        #print('*predictlabel:', self.decode(test_inputdata.dot(self.W)))
        #print('*accuracy:',show_accuracy(self.decode(test_inputdata.dot(self.W)),testlabel))
        return self.decode(test_inputdata.dot(self.W))
    
    def addWeight(self,trainlabel):
        #对BLS设置加权，按照训练集中的label比例进行权重设置
        #求训练数据集中label为1的个数
        count=0
        for z in range(len(trainlabel)):
            #print ('个数 %d ' %z)
            #print('k_train_label:', k_train_label[z])
            if trainlabel[z]==1:
                count=count+1
        #print ('label为1的个数 %d ' %j)
        #print(count)
        #print(k_train_label.shape)
        #print(len(k_train_label))
        #权重设置 
        weight = np.zeros((len(trainlabel),len(trainlabel)))
        for i in range(len(trainlabel)):
            if trainlabel[i]==1:
                #weight[i,i]=0.618/count
                weight[i,i]=2*(len(trainlabel)-count)/len(trainlabel)
            else:
                #weight[i,i]=1/(len(k_train_label)-count)
                weight[i,i]=2*count/len(trainlabel)
        #print('Weight:', weight)
        return weight

In [13]:
#将数据集划分为训练集和测试集
traindata,testdata,trainlabel,testlabel = train_test_split(X_res,y_res,test_size=0.25,random_state = 0)
#print(traindata.shape,trainlabel.shape,testdata.shape,testlabel.shape)
#X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2)
#y_train = np_utils.to_categorical(y_train)
#y_test = np_utils.to_categorical(y_test)

In [14]:
k_acc_list, k_f1_list, k_auc_list, k_recall_list, k_mcc_list, k_Gm_list=[],[],[],[],[],[]
#这里设置shuffle设置为ture就是打乱顺序在分配
kf = KFold(n_splits=3,shuffle=True,random_state=42)
for map_times in np.arange(15,36,5):
    acc_list_tmp, f1_list_tmp, auc_list_tmp, recall_list_tmp, mcc_list_tmp,Gm_list_tmp=[],[],[],[],[],[]
    for enhance_times in np.arange(15, 36,5):
        for k, (train, test) in enumerate(kf.split(traindata, trainlabel)):
            # kf.split输出的是索引，所以由索引获取交叉后的训练集和测试集及标签
            k_train_data,k_train_label = traindata[train], trainlabel[train]
            k_test_data,k_test_label = traindata[test], trainlabel[test]           

            bls = broadnet(maptimes = map_times, 
                       enhencetimes = enhance_times,
                       map_function = 'relu',
                       enhence_function = 'relu',
                       batchsize =100,
                       reg = 0.001)

            #根据训练数据设置权重
            weight=bls.addWeight(k_train_label)

            #训练
            starttime = datetime.datetime.now()
            bls.fit(k_train_data,k_train_label,weight)
            endtime = datetime.datetime.now()
            #print('the training time of BLS is {0} seconds'.format((endtime - starttime).total_seconds()))

            #print('k_test_label:', k_test_label)
            #预测
            k_predict_label = bls.predict(k_test_data)
            #print('k_predict_label:', k_predict_label)

            #评价指标计算
            acc=accuracy_score(k_test_label,k_predict_label,normalize=True)
            fmeasure=f1_score(k_test_label,k_predict_label, average='weighted', labels=np.unique(k_test_label))
            try:
                auc=roc_auc_score(k_test_label,k_predict_label, average='macro', sample_weight=None)
            except ValueError:
                pass
            recall=recall_score(k_test_label, k_predict_label, average='weighted')
            MCC=matthews_corrcoef(k_test_label,k_predict_label)
            Gmeasure=geometric_mean_score(k_test_label,k_predict_label, average='weighted')

            #将此次的十折交叉验证的结果 (10个)保存到pi_list_tmp中
            acc_list_tmp.append(acc)
            f1_list_tmp.append(fmeasure)
            auc_list_tmp.append(auc)
            recall_list_tmp.append(recall)
            mcc_list_tmp.append(MCC)
            Gm_list_tmp.append(Gmeasure)

        #求平均保存到k_acc_list中   
        k_average_acc=np.mean(acc_list_tmp)
        k_average_acc=round(k_average_acc,5)
        k_acc_list.append(k_average_acc)

        #求平均保存到k_f1_list中   
        k_average_f1=np.mean(f1_list_tmp)
        k_average_f1=round(k_average_f1,5)
        k_f1_list.append(k_average_f1)

        #求平均保存到k_auc_list中   
        k_average_auc=np.mean(auc_list_tmp)
        k_average_auc=round(k_average_auc,5)
        k_auc_list.append(k_average_auc)
        
        #求平均保存到k_recall_list中   
        k_average_recall=np.mean(recall_list_tmp)
        k_average_recall=round(k_average_recall,5)
        k_recall_list.append(k_average_recall)


        #求平均保存到k_mcc_list中   
        k_average_mcc=np.mean(mcc_list_tmp)
        k_average_mcc=round(k_average_mcc,5)
        k_mcc_list.append(k_average_mcc)

        #求平均保存到k_Gm_list中   
        k_average_Gm=np.mean(Gm_list_tmp)
        k_average_Gm=round(k_average_Gm,5)
        k_Gm_list.append(k_average_Gm)
        print(f'maptiems:{map_times}\tenhancetimes:{enhance_times}\tk_average_acc:{k_average_acc}\tk_average_f1:{k_average_f1}\tk_average_auc:{k_average_auc}\tk_average_recall:{k_average_recall}\tk_average_mcc:{k_average_mcc}\tk_average_Gm:{k_average_Gm}\t')

k_acc_array=np.array(k_acc_list)
k_acc_array=k_acc_array.reshape(5,5)
# 一维最大值索引
#idx_max_ravel = np.argmax(k_acc_array)
# true索引
#idx_max = np.unravel_index(idx_max_ravel, k_acc_array.shape)
#print('max times:',idx_max)

k_f1_array=np.array(k_f1_list)
k_f1_array=k_f1_array.reshape(5,5)

k_auc_array=np.array(k_auc_list)
k_auc_array=k_auc_array.reshape(5,5)

k_recall_array=np.array(k_recall_list)
k_recall_array=k_recall_array.reshape(5,5)

k_mcc_array=np.array(k_mcc_list)
k_mcc_array=k_mcc_array.reshape(5,5)

k_Gm_array=np.array(k_Gm_list)
k_Gm_array=k_Gm_array.reshape(5,5)

sio.savemat('./data_remember/0.75WBLS_Smote_PC1_MEbest.mat',{'acc':k_acc_array,'f1':k_f1_array,'auc':k_auc_array,'recall':k_recall_array,'mcc':k_mcc_array,'Gm':k_Gm_array})

maptiems:15	enhancetimes:15	k_average_acc:0.86481	k_average_f1:0.86444	k_average_auc:0.8642	k_average_recall:0.86481	k_average_mcc:0.73272	k_average_Gm:0.8642	
maptiems:15	enhancetimes:20	k_average_acc:0.86578	k_average_f1:0.86543	k_average_auc:0.86542	k_average_recall:0.86578	k_average_mcc:0.73479	k_average_Gm:0.86542	
maptiems:15	enhancetimes:25	k_average_acc:0.86223	k_average_f1:0.86187	k_average_auc:0.86183	k_average_recall:0.86223	k_average_mcc:0.72749	k_average_Gm:0.86183	
maptiems:15	enhancetimes:30	k_average_acc:0.86206	k_average_f1:0.86168	k_average_auc:0.86178	k_average_recall:0.86206	k_average_mcc:0.7277	k_average_Gm:0.86178	
maptiems:15	enhancetimes:35	k_average_acc:0.86377	k_average_f1:0.86338	k_average_auc:0.86347	k_average_recall:0.86377	k_average_mcc:0.73138	k_average_Gm:0.86347	
maptiems:20	enhancetimes:15	k_average_acc:0.86028	k_average_f1:0.85977	k_average_auc:0.86037	k_average_recall:0.86028	k_average_mcc:0.7261	k_average_Gm:0.86037	
maptiems:20	enhancetimes:20	k_av